# TAQ data using WRDS API

## TOC:
* [Setup](#bullet1)
* [WRDS API](#bullet2)
* [Data analysis](#bullet3)
* [Converting raw data to daily data](#bullet4)


This part contains the code for calling the WRDS API. I will be using this API to collect the TAQ data. This method is easier, as some data can already be filtered using filters like exchange code. This will decrease the data size.

### Setup <a class="anchor" id="bullet1"></a>

In [ ]:
!pip install wrds
!pip install numpy

In [2]:
import wrds
import time
import pandas as pd
import os
import numpy as np
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 20)

In [ ]:
db = wrds.Connection()
# create_pgpass_file()

In [14]:
def track_runtime(func):
    '''Decorator that reports the execution time.'''
  
    def wrap(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        runtime = round(end-start, 2)
        print(f"Function [{func.__name__}] finished with runtime [{runtime}] seconds")
        return result
    return wrap

### WRDS API <a class="anchor" id="bullet2"></a>

Now the API has been setup, we can call a couple of functions to explore it. 
- `db.list_libraries()` calls all available libraries which are currently available. I will be using the `taqmsec` library.
- The `taqmsec` library consists out of vast number of tables. Each table contains data for a different period. I am using the `ctm` datatable and will use the table for each relevant year. 

In [ ]:
db.list_libraries()

In [42]:
table_list = [item for item in db.list_tables(library='taqmsec') if 'ctm' in item and len(item) < 9]

table_list

['ctm_2003',
 'ctm_2004',
 'ctm_2005',
 'ctm_2006',
 'ctm_2007',
 'ctm_2008',
 'ctm_2009',
 'ctm_2010',
 'ctm_2011',
 'ctm_2012',
 'ctm_2013',
 'ctm_2014',
 'ctm_2015',
 'ctm_2016',
 'ctm_2017',
 'ctm_2018',
 'ctm_2019',
 'ctm_2020',
 'ctm_2021',
 'ctm_2022',
 'ctm_2023']

In [27]:
# db.describe_table(library='taqmsec', table='ctm_2018')
db.describe_table(library='taqmsec', table='ctm_20180102')


Approximately 9196317945 rows in taqmsec.ctm_2018.


,name,nullable,type,comment
0,date,True,DATE,None
1,time_m,True,TIME,None
2,time_m_nano,True,SMALLINT,None
3,ex,True,VARCHAR(1),None
4,sym_root,True,TEXT,None
5,sym_suffix,True,TEXT,None
6,tr_scond,True,VARCHAR(4),None
7,size,True,INTEGER,None
8,price,True,NUMERIC,None
9,tr_stop_ind,True,VARCHAR(1),None


In [10]:
database = 'taqmsec'
table = 'ctm_2018'

ticker = 'TSLA'
exchange_code = 'D'
start_date = '2018-01-15'
end_date = '2018-01-31'
df = db.raw_sql(f"select date, time_m, ex, sym_root, size, price from {database}.{table} where sym_root in ('{ticker}') and ex = '{exchange_code}' and date > '{start_date}' and date < '{end_date}'")
# df = db.raw_sql("select date, time_m, ex, sym_root, size, price from taqmsec.ctm_2018 where sym_root in ('GME', 'AMD', 'AAPL') and size > 100000 and date < '2018-01-31'")
df.head()

,date,time_m,ex,sym_root,size,price
0,2018-01-19,08:00:00.018172,D,TSLA,700,347.25
1,2018-01-19,08:00:03.706403,D,TSLA,25,346.19
2,2018-01-19,08:00:08.348072,D,TSLA,100,347.32
3,2018-01-19,08:00:08.349734,D,TSLA,100,346.50
4,2018-01-19,08:00:08.364496,D,TSLA,1000,347.32


This is the function that actually calls the API. 

In [15]:
@track_runtime
def call_taq_api(ticker, year=2018, database='taqmsec', exchange_code = 'D', start_date = '2018-05-01', end_date = '2020-08-31'):
    
    # Call API
    print(f'Calling API for ticker [{ticker}], year [{year}]')
    df = db.raw_sql(f"select date, time_m, ex, sym_root, size, price from {database}.ctm_{year} where sym_root in ('{ticker}') and ex = '{exchange_code}' and date >= '{start_date}' and date <= '{end_date}'")
    
    # Check number of transactions found
    transaction_count = len(df.index)
    print(f"Found [{transaction_count}] rows of data for [{ticker}]")
    
    return df


In [16]:
ticker_list = ['TSLA', 'GME', 'LULU', 'ATVI', 'IQ', 'WMT', 'SBUX', 'F', 'TLRY', 'BAC', 'SHOP','MU', 'SNAP', 'AMD', 'DIS', 'SQ', 'BABA', 'V', 'BA', 'NVDA', 'GE', 'AAPL', 'AMZN', 'MSFT', 'NFLX']
taq_path = r"E:/Users/Christiaan/Large_Files/Thesis/taq/unfiltered"
# ticker_list = ['GME']

def gather_taq(save=False):
    if save:
        for ticker in ticker_list:
            # Create file to save output
            file_loc = os.path.join(taq_path, f"{ticker}.csv").replace('\\', '/')
            
            # Check if file already exists and skips API request if file exists
            if os.path.isfile(file_loc):
                print(f"Skipping TAQ API request for ticker [{ticker}] as file exists: [{file_loc}]")
                continue
            
            # Looping through the years
            year_list = [2018, 2019, 2020]
            for year in year_list:
                
                # Calling api
                df = call_taq_api(ticker, year)

                # If file exists, append
                if os.path.isfile(file_loc):
                    print("file exits, appending")
                    df.to_csv(file_loc, mode='a', header=False)
                    print(f"Saving output at: {file_loc}\n")
                
                # Else create new file
                else:
                    print(f"file does not exist exits, saving file at: {file_loc}")
                    df.to_csv(file_loc, encoding='utf-8')

                
                # Cooling down API
                print("Sleeping for 2 secs")
                time.sleep(2)
                
            
            print(df)
            
            # Cooling down API
            print("Sleeping for 10 secs")
            time.sleep(10)


gather_taq(save=True)

Calling API for ticker [TSLA], year [2018]
Found [5087172] rows of data for [TSLA]
Function [call_taq_api] finished with runtime [51.16] seconds
file does not exist exits, saving file at: E:/Users/Christiaan/Large_Files/Thesis/taq/unfiltered/TSLA.csv
Sleeping for 2 secs
Calling API for ticker [TSLA], year [2019]
Found [7591145] rows of data for [TSLA]
Function [call_taq_api] finished with runtime [63.52] seconds
file exits, appending
Saving output at: E:/Users/Christiaan/Large_Files/Thesis/taq/unfiltered/TSLA.csv

Sleeping for 2 secs
Calling API for ticker [TSLA], year [2020]
Found [27730855] rows of data for [TSLA]
Function [call_taq_api] finished with runtime [357.79] seconds
file exits, appending
Saving output at: E:/Users/Christiaan/Large_Files/Thesis/taq/unfiltered/TSLA.csv

Sleeping for 2 secs
              date           time_m ex sym_root  size     price
0       2020-01-21  08:00:00.597186  D     TSLA     7  507.5100
1       2020-01-21  08:00:00.814201  D     TSLA   317  507.59

Sleeping for 2 secs
Calling API for ticker [SBUX], year [2019]
Found [3304968] rows of data for [SBUX]
Function [call_taq_api] finished with runtime [23.67] seconds
file exits, appending
Saving output at: E:/Users/Christiaan/Large_Files/Thesis/taq/unfiltered/SBUX.csv

Sleeping for 2 secs
Calling API for ticker [SBUX], year [2020]
Found [4760216] rows of data for [SBUX]
Function [call_taq_api] finished with runtime [34.3] seconds
file exits, appending
Saving output at: E:/Users/Christiaan/Large_Files/Thesis/taq/unfiltered/SBUX.csv

Sleeping for 2 secs
              date           time_m ex sym_root  size  price
0       2020-01-21  08:00:01.191476  D     SBUX    20  93.40
1       2020-01-21  08:00:01.233447  D     SBUX    20  93.09
2       2020-01-21  08:00:01.281050  D     SBUX    11  93.40
3       2020-01-21  08:00:01.485403  D     SBUX    25  93.40
4       2020-01-21  08:00:01.525933  D     SBUX    12  93.15
...            ...              ... ..      ...   ...    ...
260211  2020-07-

Calling API for ticker [MU], year [2020]
Found [5624528] rows of data for [MU]
Function [call_taq_api] finished with runtime [38.77] seconds
file exits, appending
Saving output at: E:/Users/Christiaan/Large_Files/Thesis/taq/unfiltered/MU.csv

Sleeping for 2 secs
              date           time_m ex sym_root  size    price
0       2020-01-21  08:00:00.535066  D       MU   100  57.2000
1       2020-01-21  08:00:00.658349  D       MU   400  57.3075
2       2020-01-21  08:00:01.629656  D       MU   100  57.1899
3       2020-01-21  08:00:02.322959  D       MU   300  57.2000
4       2020-01-21  08:00:02.431546  D       MU     5  57.2000
...            ...              ... ..      ...   ...      ...
124523  2020-07-14  19:49:15.007509  D       MU   500  49.9400
124524  2020-07-14  19:51:03.214898  D       MU    50  49.9500
124525  2020-07-14  19:51:03.215538  D       MU   200  49.9500
124526  2020-07-14  19:52:16.615796  D       MU    99  49.9400
124527  2020-07-14  19:57:43.019387  D      

              date           time_m ex sym_root  size     price
0       2020-01-21  08:00:01.439311  D     BABA   190  223.0100
1       2020-01-21  08:00:01.477995  D     BABA   100  223.0100
2       2020-01-21  08:00:01.484362  D     BABA   300  223.2000
3       2020-01-21  08:00:01.488205  D     BABA    39  223.2000
4       2020-01-21  08:00:01.506531  D     BABA   100  223.1500
...            ...              ... ..      ...   ...       ...
223863  2020-07-16  19:55:05.818482  D     BABA    30  242.6000
223864  2020-07-16  19:57:44.670805  D     BABA    15  243.0000
223865  2020-07-16  19:57:50.437526  D     BABA    60  243.0000
223866  2020-07-16  19:57:50.439938  D     BABA    40  243.0000
223867  2020-07-16  19:58:14.333321  D     BABA   200  242.9999

[7223868 rows x 6 columns]
Sleeping for 10 secs
Calling API for ticker [V], year [2018]
Found [2469671] rows of data for [V]
Function [call_taq_api] finished with runtime [16.24] seconds
file does not exist exits, saving file at: E

Calling API for ticker [AMZN], year [2018]
Found [7702833] rows of data for [AMZN]
Function [call_taq_api] finished with runtime [94.0] seconds
file does not exist exits, saving file at: E:/Users/Christiaan/Large_Files/Thesis/taq/unfiltered/AMZN.csv
Sleeping for 2 secs
Calling API for ticker [AMZN], year [2019]
Found [8312805] rows of data for [AMZN]
Function [call_taq_api] finished with runtime [68.71] seconds
file exits, appending
Saving output at: E:/Users/Christiaan/Large_Files/Thesis/taq/unfiltered/AMZN.csv

Sleeping for 2 secs
Calling API for ticker [AMZN], year [2020]
Found [12174093] rows of data for [AMZN]
Function [call_taq_api] finished with runtime [97.07] seconds
file exits, appending
Saving output at: E:/Users/Christiaan/Large_Files/Thesis/taq/unfiltered/AMZN.csv

Sleeping for 2 secs
              date           time_m ex sym_root  size    price
0       2020-01-21  08:00:00.514387  D     AMZN    10  1857.50
1       2020-01-21  08:00:01.436009  D     AMZN     5  1857.00
2 

### Data analysis <a class="anchor" id="bullet3"></a>

In [6]:
def loop_tickers(func):
    '''Decorator that loops all (ticker) files in the directory.'''
  
    def wrap(*args, **kwargs):
        if 'file_dir' in kwargs:
            for filename in os.listdir(filedir):
                csv_path = os.path.join(filedir, filename)
                kwargs['ticker'] = csv_path.split("\\")[-1].split(".")[0]
                # checking if it is a file
                if os.path.isfile(csv_path):
                    
                    message = f"""Now executing function {func.__name__} for [{filename}], variables:
                    - csv_path: [{csv_path}]
                    - kwargs: [{kwargs}])"""
                    print(message)

                    result = func(csv_path=csv_path, *args, **kwargs)
                    
        else:
            raise ValueError("Wrapper can not loop, as no file directory given. Please specify which folder needs to be looped by setting 'file_dir' variable.") 
        
        
        
        return result
    return wrap

In [ ]:
result_dict = {}

@loop_tickers
def count_rows(csv_path, *args, **kwargs):
    ticker = csv_path.split("\\")[-1].split(".")[0]
    
    # Setting start count and chunk size
    row_count = 0
    chunksize = 10 ** 6
    
    # Start counting the rows in the csv, chunk by chunk
    for chunk in pd.read_csv(csv_path, chunksize=chunksize):
        chunk_rows = len(chunk.index)
        row_count = row_count + chunk_rows
        print(f"Count for [{ticker}] increase by [{chunk_rows}] this chunk")

    # Save results
    print(f"Saving results for [{ticker}], found [{row_count}] total rows\n")
    result_dict[f"{ticker}"] = row_count

    return row_count


filedir = r"E:/Users/Christiaan/Large_Files/Thesis/taq/unfiltered"
count_rows(file_dir= filedir)

ticker_DA = pd.DataFrame(result_dict.items(), columns=['ticker', 'obs_count'])
ticker_DA

**Observation count** 

A total of `504,379,931` different transaction from the TAQ database are found. The Apple (APPL) stock had the most transaction, with a total of `54,052,875` while GameStop (GME) had the least amount of transactions: `3,148,072`.

In [116]:
ticker_DA


,ticker,obs_count
0,AAPL,54052875
1,AMD,43316885
2,AMZN,28189731
3,ATVI,7212405
4,BA,27336651
5,BABA,20543175
6,BAC,25863085
7,DIS,19156523
8,F,22274196
9,GE,30549569


### Converting raw data to daily data <a class="anchor" id="bullet4"></a>

In [10]:
def filters(df):
    #     -----------------     Filtering data     -----------------
    # Removing all observations where the Exchange code (EX) is not 'D'
    df = df[df['ex'] == 'D']

    # Creating a new column which contains the sub penny prices
    df['sub_penny'] = (df['price'] - (df['price'] * 100).apply(np.floor) / 100).round(4)

    # Removing all sub penny prices which are a round penny (i.e. equal to 0.0000 or 0.0010)
    df = df[~(df['sub_penny'] == 0.0000)]
    df = df[~(df['sub_penny'] > 0.0099)]

    # Removing all sub penny prices which are in the 0.4 - 0.6 range
    df = df[(df['sub_penny'] <= 0.0040) | (df['sub_penny'] >= 0.0060)]

    #     -----------------     Creating new data     -----------------
    # Categorize a trade as buy or sell. Also buy/sell volumes
    df['buysell'] = np.where(((df['sub_penny'] > 0.0000) & (df['sub_penny'] < 0.0050)), "sell", "buy")

    # To count buys and sells, I specify 1 or 0 for both buy and sell.
    # Being a buy automatically means having a 0 for sell. This is done to easily count total trades.
    df['buy'] = np.where(df['buysell'] == "buy", 1, 0)
    df['sell'] = np.where(df['buysell'] == "sell", 1, 0)

    # Next, I specify whether the volume is buy or sell.
    df['buy_vol'] = np.where(df['buysell'] == "buy", df['size'], 0)
    df['sell_vol'] = np.where(df['buysell'] == "sell", df['size'], 0)

    #     -----------------     Converting to daily data new data     -----------------
    # Taking daily sums and averages for the columns. The dictionary indicates which one is taken.
    df = (df.groupby('date').agg(
        {'price': 'mean', 'size': 'sum', 'buy': 'sum', 'sell': 'sum', 'buy_vol': 'sum', 'sell_vol': 'sum'})
          .round(2)
          .rename(columns={'size': 'total_vol'}))

    # Resetting 'date' index and converting it to datetime
    df = df.reset_index()
    df['date'] = pd.to_datetime(df['date'].astype(str), format='%Y-%m-%d')

    # Creating total amount of traders, total volume and total amount of money traded
    df['total_bs'] = df['buy'] + df['sell']
    df['total_vol'] = df['buy_vol'] + df['sell_vol']
    df['total_price'] = df['price'] * df['total_vol']
    
    rolling_window = 5
    df['bs_change'] = (df['buy'] - df['sell']) / df['total_bs'].rolling(rolling_window).mean()
    df['vol_change'] = (df['buy_vol'] - df['sell_vol']) / df['total_vol'].rolling(rolling_window).mean()
    df['bs_change'] = df['bs_change'].round(4)
    df['vol_change'] = df['vol_change'].round(4)
    
    return df
    

In [ ]:
@loop_tickers
def filter_data(csv_path, *args, **kwargs):
    """This function filters the unfiltered data to daily data"""
    df = pd.read_csv(csv_path)
    
    df = filters(df)

    # Setup save location by changing base_path from /unfiltered to /filtered
    base_path = os.path.dirname(kwargs['file_dir'])
    base_path = os.path.join(base_path, 'filtered').replace('\\', '/')
    
    ticker = kwargs['ticker']
    save_path = os.path.join(base_path, f"{ticker}.csv").replace('\\', '/')

    # Save results
    print(df)
    df.to_csv(save_path, encoding='utf-8')


filedir = r"E:/Users/Christiaan/Large_Files/Thesis/taq/unfiltered"
filter_data(file_dir= filedir)



In [ ]:
@track_runtime
def functionn(n):
    '''Counts down'''
    for i in range(n):
        time.sleep(1)
    print("done with function")
  
functionn(2)